In [11]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langgraph.store.memory import InMemoryStore
from langgraph.utils.config import get_store 
from langmem import (
    # 让智能体创建、更新和删除记忆 
    create_manage_memory_tool,
)

In [12]:
def prompt(state):
    """为LLM准备消息。"""
    # 从配置的上下文变量中获取存储; 
    store = get_store() # 与提供给 `create_react_agent` 的相同
    memories = store.search(
        # 在与我们为智能体配置的相同命名空间内搜索
        ("memories",),
        query=state["messages"][-1].content,
    )
    system_msg = f"""You are a helpful assistant.

## Memories
<memories>
{memories}
</memories>
"""
    return [{"role": "system", "content": system_msg}, *state["messages"]]

In [13]:
from langchain import embeddings
from langchain_openai import OpenAIEmbeddings
embedding=OpenAIEmbeddings(
    api_key="sk-081025394d9e4f35adbb67ea07c3dae1", 
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    model="text-embedding-v4",
    check_embedding_ctx_length = False,
    dimensions=1536 
)
store = InMemoryStore(
    index={ # 存储提取的记忆 
        "dims": 1536,
        "embed": embedding,
    }
) 

In [14]:
checkpointer = MemorySaver() # 检查点图状态 

In [15]:
from langchain_openai import ChatOpenAI
model_qwen=ChatOpenAI(
    api_key="sk-081025394d9e4f35adbb67ea07c3dae1", 
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    model="qwen3-30b-a3b-instruct-2507",
)

In [16]:
result=model_qwen.invoke("你好")
print(result.content)

你好！有什么我可以帮你的吗？😊


In [17]:
agent = create_react_agent( 
    model=model_qwen,
    prompt=prompt,
    tools=[ # 添加记忆工具 
        # 智能体可以调用 "manage_memory" 来
        # 通过ID创建、更新和删除记忆
        # 命名空间为记忆添加作用域。要
        # 为每个用户限定记忆范围，使用 ("memories", "{user_id}"): 
        create_manage_memory_tool(namespace=("memories",)),
    ],
    # 我们的记忆将存储在这个提供的BaseStore实例中
    store=store,
    # 图的"状态"将在每个节点完成执行后进行检查点
    # 用于跟踪聊天历史和持久执行
    checkpointer=checkpointer, 
)

In [20]:
agent.get_graph().draw_mermaid_png(output_file_path="agent.png")


b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\xd8\x00\x00\x00\xf9\x08\x02\x00\x00\x00\xca\xb2l)\x00\x00\x00\x01sRGB\x00\xae\xce\x1c\xe9\x00\x00 \x00IDATx\x9c\xed\x9dw\\\x14\xd7\xfe\xbf\xcflo\xb4-t\x10,\x88\x88\x8a\x015\x12c\xc3\x98`D\xc5\xden,\xbf\\\xbd\x96$$\x1a\xef5\xb9\xc6\xdc\xe4\xab\xc6\x1bs5\xa2\xd1`\xf4F\x89\x8a\x05\xc4\x1eE\x13AA"\xa8\xa0\x14AAPz[\xb6\xf7\x9d\xdf\x1f\xeb\x8bpq\xa9\xee\xec\x9ce\xcf\xf3\xca\x1f\xbb;\xb3s\xde\x1b\x1e\xcf|\xe6\xcc\xcc\x19\x0c\xc7q\x80@\x90\r\x85\xec\x00\x08\x04@""`\x01\x89\x88\x80\x02$"\x02\n\x90\x88\x08(@""\xa0\x80Fv\x00\xe8\xd0\xaa\r\r\x95Z\xa5\xcc\xa0\x94\xe9\rz\\\xa7\xb5\x81\xe1-&\x9bBc`\x1c\x07\x1a\xc7\x81\xe2\xe6\xcb&;NO\xc0\xd08\xa2\t\xa5\\\xff\xf8\xae\xbc4O\xd1T\xa3qvep\x1c\xa8\x1c\x07\x9a#\x9f\xa6\xd3\xd8\xc0\xff\x1f:\x8b"\xae\xd1*ez\x1a\x03+/T\xf6\r\xe6\xf5\x1d\xca\xed7\x94Gv\xaen\x80D\x048\x8eg\x9co\xac)S\x89|X}\x83\xb9\xde\x038d\'z%\xb4jci\x9e\xfcy\x91\xaa\xf2\x89*<J\x10\xf0\x9a\x03\xd9\x89\xba\x84\xbd\x8bX\xf8\x87\xf4zB]x\x94\xe0\x

In [25]:
config = {"configurable": {"thread_id": "thread-a"}} 
 
# 使用智能体。智能体还没有保存任何记忆，
# 所以它不了解我们
response = agent.invoke( 
    { 
        "messages": [ 
            {"role": "user", "content": "你知道我喜欢哪种显示模式吗?"}
        ] 
    }, 
    config=config, 
) 
print(response["messages"][-1].content) 

是的，我知道！你偏好黑色显示模式。我会在后续交互中保持这一设置。


In [26]:
agent.invoke( 
    { 
        "messages": [ 
            {"role": "user", "content": "黑色，记住他"} 
        ] 
    }, 
    # 我们将通过使用具有相同thread_id的config
    # 来继续对话(thread-a)
    config=config, 
) 
print(response["messages"][-1].content) 

是的，我知道！你偏好黑色显示模式。我会在后续交互中保持这一设置。


In [ ]:
# 新线程 = 新对话！
new_config = {"configurable": {"thread_id": "thread-b"}} 
# 智能体只能回忆起
# 它使用manage_memories工具明确保存的内容
response = agent.invoke( 
    {"messages": [{"role": "user", "content": "你好。你还记得我吗？你知道我有什么偏好吗？"}]},
    config=new_config, 
) 
print(response["messages"][-1].content) 


你好！虽然我无法记住你作为个体的详细信息，但我可以访问一些关于你的偏好信息。根据之前的记录，我知道你偏好使用黑色显示模式。如果你还有其他偏好或希望我记住什么，请告诉我，我会帮你记录下来。


In [32]:
config = {"configurable": {"thread_id": "thread-a"}} 
 
# 使用智能体。智能体还没有保存任何记忆，
# 所以它不了解我们
response = agent.invoke( 
    { 
        "messages": [ 
            {"role": "user", "content": "我不叫张熙浚，我叫严昊然"}
        ] 
    }, 
    config=config, 
) 
print(response["messages"][-1].content) 

已更新记忆！你的名字是**严昊然**。非常抱歉之前的混淆，很高兴现在正确地认识你了，严昊然！


In [36]:
# 新线程 = 新对话！
new_config = {"configurable": {"thread_id": "thread-c"}} 
# 智能体只能回忆起
# 它使用manage_memories工具明确保存的内容
response = agent.invoke( 
    {"messages": [{"role": "user", "content": "你一般会在什么时候选择调用记忆工具阿"}]},
    config=new_config, 
) 
print(response["messages"][-1].content) 

我会在以下几种情况下主动调用记忆工具来管理记忆：

1. **发现新的用户偏好**：比如你提到喜欢某种颜色、格式或特定的交互方式。
2. **收到明确的记忆请求**：当你要求我记住某件事，比如“请记住我喜欢蓝色”。
3. **记录重要上下文**：在对话中涉及关键信息时，为了保持连贯性而保存。
4. **修正过时或错误的信息**：如果我发现之前记录的内容不准确，会及时更新或删除。

这样可以确保我能更好地理解你，并提供更个性化的帮助。


In [39]:
response = agent.invoke( 
    {"messages": [{"role": "user", "content": "带我学习langchain"}]},
    config=new_config, 
) 
print(response["messages"][-1].content) 

好的，张熙浚！我已经记住了您正在学习 **LangChain**。这是一套强大的框架，用于构建基于大语言模型（LLM）的应用程序。

为了更好地帮助您，我们可以从以下几个方向开始：

1.  **基础概念**：了解 LangChain 的核心组件，如 `Models`、`Prompts`、`Chains` 和 `Agents`。
2.  **动手实践**：通过一个简单的例子，比如用 LLM 做问答或摘要，来感受它的魅力。
3.  **深入探索**：学习如何将 LangChain 与您的数据（如文档、数据库）结合，构建更复杂的应用。

您想先从哪个部分开始呢？或者您对 LangChain 的哪个具体方面最感兴趣？


In [ ]:
# 新线程 = 新对话！
new_config = {"configurable": {"thread_id": "thread-d"}} 
# 智能体只能回忆起
# 它使用manage_memories工具明确保存的内容
response = agent.invoke( 
    {"messages": [{"role": "user", "content": "你是如何存储记忆的"}]},
    config=new_config, 
) 
print(response["messages"][-1].content) 

根据我记忆中的信息，您在不同时间点提到了两个名字：

*   **张熙浚**：这是在 `2025-08-14T08:21:03.724251+00:00` 这个时间点记录的。
*   **严昊然**：这是在 `2025-08-14T08:22:52.913889+00:00` 这个时间点记录的。

这两个名字是相互矛盾的。为了确保我能准确地称呼您，能否请您确认一下，您希望我称呼您为 **张熙浚** 还是 **严昊然**？
